<a href="https://colab.research.google.com/github/smjoaopaulo/Desafio-TOTVS-Engenheiro-de-Dados-Jr./blob/main/Desafio_TOTVS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [3]:
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Desafio TOTVS") \
    .getOrCreate()

In [5]:
spark = SparkSession.builder \
    .appName("Exemplo PySpark") \
    .getOrCreate()

In [6]:
file_path = '/content/dados_jogos.csv'

In [42]:
dados_jogos = spark.read.csv(file_path, header=True, inferSchema=True, sep=',')
dados_jogos.show()

+---+----+--------------------+--------------------+--------------------+--------------------+-------------+------------+--------------------+--------------------+------------+--------------------+------------+
|_c0|  id|               title|           thumbnail|   short_description|            game_url|        genre|    platform|           publisher|           developer|release_date|         profile_url|release_year|
+---+----+--------------------+--------------------+--------------------+--------------------+-------------+------------+--------------------+--------------------+------------+--------------------+------------+
|  0|1136|         Overwatch 2|https://www.mmobo...|Big changes come ...|https://www.mmobo...|      Shooter|PC (Windows)|Activision Blizza...|Blizzard Entertai...|  2022-10-04|https://www.mmobo...|        2022|
|  1| 523|            Lost Ark|https://www.mmobo...|Journey throughou...|https://www.mmobo...|         ARPG|PC (Windows)|        Amazon Games|           Smi

In [43]:
dados_jogos_apos2008 = dados_jogos.filter(dados_jogos['release_year'] > 2008)
dados_jogos_apos2008.show()


+---+----+--------------------+--------------------+--------------------+--------------------+-------------+------------+--------------------+--------------------+------------+--------------------+------------+
|_c0|  id|               title|           thumbnail|   short_description|            game_url|        genre|    platform|           publisher|           developer|release_date|         profile_url|release_year|
+---+----+--------------------+--------------------+--------------------+--------------------+-------------+------------+--------------------+--------------------+------------+--------------------+------------+
|  0|1136|         Overwatch 2|https://www.mmobo...|Big changes come ...|https://www.mmobo...|      Shooter|PC (Windows)|Activision Blizza...|Blizzard Entertai...|  2022-10-04|https://www.mmobo...|        2022|
|  1| 523|            Lost Ark|https://www.mmobo...|Journey throughou...|https://www.mmobo...|         ARPG|PC (Windows)|        Amazon Games|           Smi

In [44]:
from pyspark.sql.functions import when, col

In [45]:
dados_jogos = dados_jogos.withColumn('antiguidade',when(col('release_date').substr(1, 4).cast('int') < 2008, 'antigo').otherwise('novo'))

In [46]:
dados_jogos.show()

+---+----+--------------------+--------------------+--------------------+--------------------+-------------+------------+--------------------+--------------------+------------+--------------------+------------+-----------+
|_c0|  id|               title|           thumbnail|   short_description|            game_url|        genre|    platform|           publisher|           developer|release_date|         profile_url|release_year|antiguidade|
+---+----+--------------------+--------------------+--------------------+--------------------+-------------+------------+--------------------+--------------------+------------+--------------------+------------+-----------+
|  0|1136|         Overwatch 2|https://www.mmobo...|Big changes come ...|https://www.mmobo...|      Shooter|PC (Windows)|Activision Blizza...|Blizzard Entertai...|  2022-10-04|https://www.mmobo...|        2022|       novo|
|  1| 523|            Lost Ark|https://www.mmobo...|Journey throughou...|https://www.mmobo...|         ARPG|

In [47]:
from pyspark.sql.functions import col, count

In [48]:
genre_count = dados_jogos.groupBy('genre').agg(count('genre').alias('count'))

In [49]:
genre_count = genre_count.orderBy(col('count').desc())
genre_count.show(1)

+------+-----+
| genre|count|
+------+-----+
|MMORPG|  153|
+------+-----+
only showing top 1 row



In [50]:
dados_jogos_NCSOFT = dados_jogos.filter(dados_jogos['publisher'] == 'NCSOFT')
dados_jogos_NCSOFT.show()

+---+---+-----+--------------------+--------------------+--------------------+------+------------+---------+---------+------------+--------------------+------------+-----------+
|_c0| id|title|           thumbnail|   short_description|            game_url| genre|    platform|publisher|developer|release_date|         profile_url|release_year|antiguidade|
+---+---+-----+--------------------+--------------------+--------------------+------+------------+---------+---------+------------+--------------------+------------+-----------+
|353|254| AION|https://www.mmobo...|Aion: Ascension i...|https://www.mmobo...|MMORPG|PC (Windows)|   NCSOFT|   NCSOFT|  2008-11-25|https://www.mmobo...|        2008|       novo|
+---+---+-----+--------------------+--------------------+--------------------+------+------------+---------+---------+------------+--------------------+------------+-----------+



In [51]:
dados_jogos_antigos = dados_jogos.orderBy(col('release_date').asc())
dados_jogos_antigos.show(4)

+--------------+-------------+----------------+--------------------+--------------------+--------------------+--------+------------+---------+---------+------------+--------------------+------------+-----------+
|           _c0|           id|           title|           thumbnail|   short_description|            game_url|   genre|    platform|publisher|developer|release_date|         profile_url|release_year|antiguidade|
+--------------+-------------+----------------+--------------------+--------------------+--------------------+--------+------------+---------+---------+------------+--------------------+------------+-----------+
|           404|          405|Pocket Starships|https://www.mmobo...|Blast off into ad...|https://www.mmobo...|Strategy| Web Browser|     Spyr|     Spyr|        null|https://www.mmobo...|        null|       novo|
|           250|          169|      Dirty Bomb|https://www.mmobo...|Dirty Bomb, forme...|https://www.mmobo...| Shooter|PC (Windows)|        "|     null|

In [52]:
titles_per_platform = dados_jogos.groupBy('platform').agg(count('title').alias('num_titles'))
titles_per_platform = titles_per_platform.orderBy(col('num_titles').desc())

In [53]:
titles_per_platform.show()

+--------------------+----------+
|            platform|num_titles|
+--------------------+----------+
|        PC (Windows)|       313|
|         Web Browser|        72|
|PC (Windows), Web...|        11|
|              MMORPG|         3|
|                null|         1|
|             Shooter|         1|
|   or more difficult|         1|
| you'll build a d...|         1|
|            Strategy|         1|
| and you'll need ...|         1|
|https://www.mmobo...|         1|
+--------------------+----------+

